# Laboratorio 4 - Pipeline
Andrea Cecilia Rivas Castañeda
16001120

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from feature_engine.encoding import OneHotEncoder

from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

In [2]:
data = pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.drop(['PassengerId', 'Cabin', 'Name'], axis = 1, inplace=True)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,1,female,35.0,1,0,113803,53.1000,S
4,0,3,male,35.0,0,0,373450,8.0500,S


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['Survived'], axis=1),
        data['Survived'],
        test_size=0.3,
        random_state=2022)

In [5]:
# TRATAMIENTO DE NAs
CATEGORICAL_VARS_WITH_NA_MISSING = ['Embarked']

NUMERICAL_VARS_WITH_NA = ['Age']

# TRANSFORMACIÓN DE VARS CATEGÓRICAS
OH_ENCODING_VARS = ['Sex']

FREQUENCY_ENCODING_VARS = ['Ticket']

CAT_ORDINAL_ENCODING_VARS = ['Embarked']

# TRANSFORMACIÓN DE VARS NUMERICAS
#Variables para binarización por sesgo fuerte
BINARIZE_VARS = ['SibSp', 'Parch']

NJ_TRANSFORM = ['Age']
LOG_TRANSFORM = ['Fare']

# VARIABLES SELECCIONADAS
FEATURES = ['Pclass', 'Sex']

In [6]:
survivedTitanic_pipeline = Pipeline([
    # IMPUTACIONES
    # Categoricas
    ('missing_imputation', 
         CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    ),
    # Numericas
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    
    #4. Imputación de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    )

])

In [7]:
survivedTitanic_pipeline.fit(X_train, y_train)

Pipeline(steps=[('missing_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['Embarked'])),
                ('missing_indicator', AddMissingIndicator(variables=['Age'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Age']))])

In [8]:
survivedTitanic_pipeline.transform(X_test)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Age_na
770,3,male,24.000000,0,0,345781,9.5000,S,0
178,2,male,30.000000,0,0,250653,13.0000,S,0
786,3,female,18.000000,0,0,3101265,7.4958,S,0
159,3,male,29.305332,8,2,CA. 2343,69.5500,S,1
656,3,male,29.305332,0,0,349223,7.8958,S,1
...,...,...,...,...,...,...,...,...,...
693,3,male,25.000000,0,0,2672,7.2250,C,0
79,3,female,30.000000,0,0,364516,12.4750,S,0
71,3,female,16.000000,5,2,CA 2144,46.9000,S,0
503,3,female,37.000000,0,0,4135,9.5875,S,0
